In [1]:
import os
os.chdir("/home/atulgang/Thesis")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-07 02:29:21,756 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-05-07 02:29:44,525 : INFO : loading Dictionary object from dict/lem10
2018-05-07 02:29:44,865 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-05-07 02:29:45,014 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-05-07 02:29:45,015 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-05-07 02:29:45,149 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus2/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [10]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus2/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [16]:
from gensim.models.doc2vec import TaggedDocument

In [14]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus2/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield TaggedDocument(itemlist, [i])

In [17]:
%%time
model = gensim.models.Doc2Vec(iter_token_corpus(), dm=0, dbow_words=1, sample=.00001, size=300, window=10, negative=5, hs=1, min_count=0, workers=8, iter=20)

2018-05-07 02:31:28,968 : INFO : collecting all words and their counts
2018-05-07 02:31:28,970 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-07 02:31:42,182 : INFO : PROGRESS: at example #10000, processed 12250127 words (927275/s), 69492 word types, 10000 tags
2018-05-07 02:31:54,486 : INFO : PROGRESS: at example #20000, processed 23019662 words (875328/s), 93100 word types, 20000 tags
2018-05-07 02:32:08,173 : INFO : PROGRESS: at example #30000, processed 33121221 words (738067/s), 108174 word types, 30000 tags
2018-05-07 02:32:18,162 : INFO : PROGRESS: at example #40000, processed 41771479 words (866030/s), 121160 word types, 40000 tags
2018-05-07 02:32:26,308 : INFO : PROGRESS: at example #50000, processed 48600564 words (838467/s), 127318 word types, 50000 tags
2018-05-07 02:32:33,782 : INFO : PROGRESS: at example #60000, processed 54839174 words (834759/s), 133012 word types, 60000 tags
2018-05-07 02:32:40,082 : INFO : PROGRESS: at example

2018-05-07 02:37:23,965 : INFO : EPOCH 1 - PROGRESS: at 1.93% examples, 81556 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:24,969 : INFO : EPOCH 1 - PROGRESS: at 1.96% examples, 82143 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:25,974 : INFO : EPOCH 1 - PROGRESS: at 1.99% examples, 82677 words/s, in_qsize 0, out_qsize 3
2018-05-07 02:37:26,977 : INFO : EPOCH 1 - PROGRESS: at 2.01% examples, 83211 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:27,995 : INFO : EPOCH 1 - PROGRESS: at 2.04% examples, 83726 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:29,020 : INFO : EPOCH 1 - PROGRESS: at 2.06% examples, 84127 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:30,030 : INFO : EPOCH 1 - PROGRESS: at 2.09% examples, 84643 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:31,034 : INFO : EPOCH 1 - PROGRESS: at 2.13% examples, 84911 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:37:32,078 : INFO : EPOCH 1 - PROGRESS: at 2.19% examples, 84954 words/s, in_qsize 2, out_qsize 0
2

2018-05-07 02:38:40,001 : INFO : EPOCH 1 - PROGRESS: at 7.23% examples, 83010 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:38:41,012 : INFO : EPOCH 1 - PROGRESS: at 7.31% examples, 82924 words/s, in_qsize 2, out_qsize 0
2018-05-07 02:38:42,044 : INFO : EPOCH 1 - PROGRESS: at 7.40% examples, 82865 words/s, in_qsize 2, out_qsize 0
2018-05-07 02:38:43,050 : INFO : EPOCH 1 - PROGRESS: at 7.51% examples, 82883 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:38:44,085 : INFO : EPOCH 1 - PROGRESS: at 7.57% examples, 82839 words/s, in_qsize 0, out_qsize 2
2018-05-07 02:38:45,089 : INFO : EPOCH 1 - PROGRESS: at 7.67% examples, 82856 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:38:46,090 : INFO : EPOCH 1 - PROGRESS: at 7.74% examples, 82827 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:38:47,124 : INFO : EPOCH 1 - PROGRESS: at 7.80% examples, 82804 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:38:48,150 : INFO : EPOCH 1 - PROGRESS: at 7.87% examples, 82721 words/s, in_qsize 4, out_qsize 0
2

2018-05-07 02:39:56,080 : INFO : EPOCH 1 - PROGRESS: at 14.00% examples, 82366 words/s, in_qsize 9, out_qsize 0
2018-05-07 02:39:57,132 : INFO : EPOCH 1 - PROGRESS: at 14.07% examples, 82328 words/s, in_qsize 16, out_qsize 0
2018-05-07 02:39:58,157 : INFO : EPOCH 1 - PROGRESS: at 14.18% examples, 82338 words/s, in_qsize 15, out_qsize 1
2018-05-07 02:39:59,166 : INFO : EPOCH 1 - PROGRESS: at 14.30% examples, 82316 words/s, in_qsize 16, out_qsize 0
2018-05-07 02:40:00,197 : INFO : EPOCH 1 - PROGRESS: at 14.45% examples, 82304 words/s, in_qsize 15, out_qsize 0
2018-05-07 02:40:01,199 : INFO : EPOCH 1 - PROGRESS: at 14.55% examples, 82323 words/s, in_qsize 14, out_qsize 0
2018-05-07 02:40:02,219 : INFO : EPOCH 1 - PROGRESS: at 14.64% examples, 82299 words/s, in_qsize 14, out_qsize 1
2018-05-07 02:40:03,218 : INFO : EPOCH 1 - PROGRESS: at 14.77% examples, 82317 words/s, in_qsize 13, out_qsize 0
2018-05-07 02:40:04,231 : INFO : EPOCH 1 - PROGRESS: at 14.87% examples, 82272 words/s, in_qsize 

2018-05-07 02:41:11,841 : INFO : EPOCH 1 - PROGRESS: at 22.43% examples, 81853 words/s, in_qsize 12, out_qsize 0
2018-05-07 02:41:12,883 : INFO : EPOCH 1 - PROGRESS: at 22.51% examples, 81843 words/s, in_qsize 15, out_qsize 0
2018-05-07 02:41:13,909 : INFO : EPOCH 1 - PROGRESS: at 22.61% examples, 81840 words/s, in_qsize 12, out_qsize 0
2018-05-07 02:41:14,954 : INFO : EPOCH 1 - PROGRESS: at 22.75% examples, 81843 words/s, in_qsize 11, out_qsize 0
2018-05-07 02:41:16,024 : INFO : EPOCH 1 - PROGRESS: at 22.85% examples, 81879 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:41:17,025 : INFO : EPOCH 1 - PROGRESS: at 22.96% examples, 81877 words/s, in_qsize 0, out_qsize 0
2018-05-07 02:41:18,026 : INFO : EPOCH 1 - PROGRESS: at 23.06% examples, 81856 words/s, in_qsize 4, out_qsize 0
2018-05-07 02:41:19,049 : INFO : EPOCH 1 - PROGRESS: at 23.16% examples, 81844 words/s, in_qsize 9, out_qsize 0
2018-05-07 02:41:20,076 : INFO : EPOCH 1 - PROGRESS: at 23.25% examples, 81840 words/s, in_qsize 8, 

...

2018-05-07 08:22:47,131 : INFO : EPOCH 20 - PROGRESS: at 83.18% examples, 81945 words/s, in_qsize 2, out_qsize 0
2018-05-07 08:22:48,159 : INFO : EPOCH 20 - PROGRESS: at 83.34% examples, 81947 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:49,201 : INFO : EPOCH 20 - PROGRESS: at 83.54% examples, 81945 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:50,208 : INFO : EPOCH 20 - PROGRESS: at 83.71% examples, 81947 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:51,212 : INFO : EPOCH 20 - PROGRESS: at 83.87% examples, 81955 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:52,231 : INFO : EPOCH 20 - PROGRESS: at 84.13% examples, 81954 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:53,233 : INFO : EPOCH 20 - PROGRESS: at 84.33% examples, 81957 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:54,271 : INFO : EPOCH 20 - PROGRESS: at 84.53% examples, 81953 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:22:55,274 : INFO : EPOCH 20 - PROGRESS: at 84.79% examples, 81959 words/s, in_qsiz

2018-05-07 08:24:02,083 : INFO : EPOCH 20 - PROGRESS: at 92.05% examples, 82094 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:24:03,110 : INFO : EPOCH 20 - PROGRESS: at 92.08% examples, 82095 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:24:04,127 : INFO : EPOCH 20 - PROGRESS: at 92.11% examples, 82098 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:24:05,234 : INFO : EPOCH 20 - PROGRESS: at 92.14% examples, 82089 words/s, in_qsize 1, out_qsize 0
2018-05-07 08:24:06,262 : INFO : EPOCH 20 - PROGRESS: at 92.17% examples, 82094 words/s, in_qsize 2, out_qsize 1
2018-05-07 08:24:07,315 : INFO : EPOCH 20 - PROGRESS: at 92.21% examples, 82101 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:24:08,323 : INFO : EPOCH 20 - PROGRESS: at 92.25% examples, 82107 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:24:09,414 : INFO : EPOCH 20 - PROGRESS: at 92.29% examples, 82104 words/s, in_qsize 2, out_qsize 0
2018-05-07 08:24:10,434 : INFO : EPOCH 20 - PROGRESS: at 92.33% examples, 82111 words/s, in_qsiz

2018-05-07 08:25:17,514 : INFO : EPOCH 20 - PROGRESS: at 94.45% examples, 82208 words/s, in_qsize 7, out_qsize 0
2018-05-07 08:25:18,534 : INFO : EPOCH 20 - PROGRESS: at 94.48% examples, 82211 words/s, in_qsize 4, out_qsize 0
2018-05-07 08:25:19,534 : INFO : EPOCH 20 - PROGRESS: at 94.50% examples, 82214 words/s, in_qsize 5, out_qsize 0
2018-05-07 08:25:20,604 : INFO : EPOCH 20 - PROGRESS: at 94.55% examples, 82215 words/s, in_qsize 2, out_qsize 0
2018-05-07 08:25:21,651 : INFO : EPOCH 20 - PROGRESS: at 94.61% examples, 82214 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:25:22,656 : INFO : EPOCH 20 - PROGRESS: at 94.65% examples, 82210 words/s, in_qsize 4, out_qsize 0
2018-05-07 08:25:23,663 : INFO : EPOCH 20 - PROGRESS: at 94.68% examples, 82216 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:25:24,665 : INFO : EPOCH 20 - PROGRESS: at 94.73% examples, 82220 words/s, in_qsize 3, out_qsize 0
2018-05-07 08:25:25,663 : INFO : EPOCH 20 - PROGRESS: at 94.78% examples, 82221 words/s, in_qsiz

2018-05-07 08:26:32,374 : INFO : EPOCH 20 - PROGRESS: at 98.60% examples, 82293 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:26:33,388 : INFO : EPOCH 20 - PROGRESS: at 98.66% examples, 82298 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:26:34,466 : INFO : EPOCH 20 - PROGRESS: at 98.73% examples, 82289 words/s, in_qsize 1, out_qsize 0
2018-05-07 08:26:35,481 : INFO : EPOCH 20 - PROGRESS: at 98.78% examples, 82301 words/s, in_qsize 2, out_qsize 0
2018-05-07 08:26:36,501 : INFO : EPOCH 20 - PROGRESS: at 98.85% examples, 82304 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:26:37,519 : INFO : EPOCH 20 - PROGRESS: at 98.94% examples, 82304 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:26:38,524 : INFO : EPOCH 20 - PROGRESS: at 99.00% examples, 82305 words/s, in_qsize 0, out_qsize 0
2018-05-07 08:26:39,537 : INFO : EPOCH 20 - PROGRESS: at 99.07% examples, 82304 words/s, in_qsize 1, out_qsize 0
2018-05-07 08:26:40,547 : INFO : EPOCH 20 - PROGRESS: at 99.14% examples, 82306 words/s, in_qsiz

CPU times: user 1d 14h 39min 26s, sys: 22min, total: 1d 15h 1min 26s
Wall time: 5h 55min 30s


In [18]:
model.save('py_models/D2V_DBOW')

2018-05-07 13:49:33,992 : INFO : saving Doc2Vec object under py_models/D2V_DBOW, separately None
2018-05-07 13:49:33,995 : INFO : storing np array 'vectors_docs' to py_models/D2V_DBOW.docvecs.vectors_docs.npy
2018-05-07 13:49:34,177 : INFO : storing np array 'vectors' to py_models/D2V_DBOW.wv.vectors.npy
2018-05-07 13:49:34,264 : INFO : storing np array 'syn1neg' to py_models/D2V_DBOW.trainables.syn1neg.npy
2018-05-07 13:49:34,352 : INFO : storing np array 'syn1' to py_models/D2V_DBOW.trainables.syn1.npy
2018-05-07 13:49:37,408 : INFO : saved py_models/D2V_DBOW


In [19]:
model = gensim.models.Word2Vec.load('py_models/D2V_DBOW')

2018-05-07 13:49:37,412 : INFO : loading Word2Vec object from py_models/D2V_DBOW
2018-05-07 13:49:38,369 : INFO : loading vocabulary recursively from py_models/D2V_DBOW.vocabulary.* with mmap=None
2018-05-07 13:49:38,370 : INFO : loading docvecs recursively from py_models/D2V_DBOW.docvecs.* with mmap=None
2018-05-07 13:49:38,371 : INFO : loading vectors_docs from py_models/D2V_DBOW.docvecs.vectors_docs.npy with mmap=None
2018-05-07 13:49:38,507 : INFO : loading wv recursively from py_models/D2V_DBOW.wv.* with mmap=None
2018-05-07 13:49:38,507 : INFO : loading vectors from py_models/D2V_DBOW.wv.vectors.npy with mmap=None
2018-05-07 13:49:38,575 : INFO : loading trainables recursively from py_models/D2V_DBOW.trainables.* with mmap=None
2018-05-07 13:49:38,576 : INFO : loading syn1neg from py_models/D2V_DBOW.trainables.syn1neg.npy with mmap=None
2018-05-07 13:49:38,645 : INFO : loading syn1 from py_models/D2V_DBOW.trainables.syn1.npy with mmap=None
2018-05-07 13:49:38,711 : INFO : loaded 

In [15]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

In [21]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [22]:
from scipy.spatial.distance import cosine
def get_scores(query):
    xx = 0
    scores = []
    i = 0
    query = model.infer_vector(query, steps=1000)
    for x in range(0,dictionary.num_docs):
        score = 1 - cosine(model.docvecs[x], query)
        scores.append(score)
    return scores

In [23]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
    print i
        

0
1
2
3
4
5
6
7
8
9
CPU times: user 2min 31s, sys: 816 ms, total: 2min 32s
Wall time: 2min 31s


In [24]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.23112709820270538, 3707],
  ['ConsumerCourt_DCDRC_106530', '0', 0.25370028614997864, 1684],
  ['ConsumerCourt_DCDRC_107608', '1', 0.33350831270217896, 97],
  ['ConsumerCourt_DCDRC_114291', '1', 0.3307918608188629, 107],
  ['ConsumerCourt_DCDRC_114382', '1', 0.3371007442474365, 88],
  ['ConsumerCourt_DCDRC_118185', '1', 0.26242348551750183, 1224],
  ['ConsumerCourt_DCDRC_130318', '0', 0.3022201359272003, 313],
  ['ConsumerCourt_DCDRC_130570', '1', 0.2240552008152008, 4624],
  ['ConsumerCourt_DCDRC_131146', '1', 0.39850568771362305, 4],
  ['ConsumerCourt_DCDRC_131717', '1', 0.35332584381103516, 45],
  ['ConsumerCourt_DCDRC_131741', '1', 0.27544888854026794, 748],
  ['ConsumerCourt_DCDRC_131818', '1', 0.4144626557826996, 2],
  ['ConsumerCourt_DCDRC_131950', '0', 0.3061183989048004, 269],
  ['ConsumerCourt_DCDRC_131972', '0', 0.3393860161304474, 82],
  ['ConsumerCourt_DCDRC_132932', '0', 0.3049383759498596, 278],
  ['ConsumerCourt_DCDRC_133592',

In [23]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.44791001081466675, 2668],
  ['ConsumerCourt_DCDRC_106530', '0', 0.2213520109653473, 23720],
  ['ConsumerCourt_DCDRC_107608', '1', 0.5074297785758972, 1197],
  ['ConsumerCourt_DCDRC_114291', '1', 0.5309199690818787, 829],
  ['ConsumerCourt_DCDRC_114382', '1', 0.5046230554580688, 1254],
  ['ConsumerCourt_DCDRC_118185', '1', 0.5567267537117004, 459],
  ['ConsumerCourt_DCDRC_130318', '0', 0.551900327205658, 518],
  ['ConsumerCourt_DCDRC_130570', '1', 0.5862826704978943, 207],
  ['ConsumerCourt_DCDRC_131146', '1', 0.5727462768554688, 301],
  ['ConsumerCourt_DCDRC_131717', '1', 0.5598759055137634, 418],
  ['ConsumerCourt_DCDRC_131741', '1', 0.4994177222251892, 1375],
  ['ConsumerCourt_DCDRC_131818', '1', 0.5408750772476196, 667],
  ['ConsumerCourt_DCDRC_131950', '0', 0.4982038140296936, 1407],
  ['ConsumerCourt_DCDRC_131972', '0', 0.5886817574501038, 191],
  ['ConsumerCourt_DCDRC_132932', '0', 0.6350814700126648, 12],
  ['ConsumerCourt_DCDRC_13359

In [25]:
model_name = 'D2V_DBOW'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [26]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [27]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)